In [3]:
import os
import sys
import numpy as np

In [4]:
import torch

In [5]:
from tensorflow import keras

In [ ]:
src_pt_model_path = "../retrained_models_after_pruned/retrained_model_20240124123209_acc_95.45455169677734_795th_epoch.pt"